Author: Joshua
Summary: Run a single instance of the two stage stochastic program <br>
Input: <br>
Output: The ambulance stationing allocation x. <br>

In [1]:
using Pkg
Pkg.add("Pkg")
Pkg.add("Dates")

           
joshpath = "C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy"
joshpath = "C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy"

PROJECT_ROOT = joshpath
currentpath = joshpath

cd(currentpath)
Pkg.activate(".")

#include(".//src//problem.jl")
#include(".//src//model.jl")

using DataFrames, Winston, JLD, CSV, Gurobi, JuMP, GLPK, Dates

   Updating registry at `C:\Users\Owner\.julia\registries\General`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`
 Activating environment at `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`


In [2]:
using AmbulanceDeployment 

In [3]:
PROJECT_ROOT * "/test/austin-data/adjacent_nbhd.csv"

"C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy/test/austin-data/adjacent_nbhd.csv"

In [5]:
# hourly calls - regions x hours x number of calls/ per region per hour (e.g. 210 x 10000 x Z)
#adjacent_nbhd - boolean matrix, regionxregion (210x210) - if neighborhoods/region are adjacent?
#coverage - boolean matrix, regions x stations (210x44) - true if station is within coverage of region
#incidents - god object

# PROJECT_ROOT = "C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy"
# adjacent_nbhd = CSV.File(PROJECT_ROOT * "/test/austin-data/adjacent_nbhd.csv") |> DataFrame
# #I CAN'T for the life of me see the difference.
adjacent_nbhd = CSV.File("C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy/test/austin_data/adjacent_nbhd.csv") |> DataFrame
coverage = CSV.File("C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy/test/austin_data/coverage.csv") |> DataFrame
#coverage = convert(Array{Bool, 2}, coverage[:, :])
hourly_calls = CSV.File("C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy/test/austin_data/Full_WeekdayCalls.csv") |> DataFrame
# weekend_hourly_calls = CSV.File("data/processed/2-weekend_calls.csv") |> DataFrame

print("\n coverage ",size(coverage))
print("\n hourly_calls ",size(hourly_calls))
print("\n adjacency ",size(adjacent_nbhd))


 coverage (209, 44)
 hourly_calls (10296, 214)
 adjacency (224, 225)

In [6]:
num_regions = 209
regions = collect(1:209)

209-element Array{Int64,1}:
   1
   2
   3
   4
   5
   6
   7
   8
   9
  10
  11
  12
  13
   ⋮
 198
 199
 200
 201
 202
 203
 204
 205
 206
 207
 208
 209

In [7]:
#reformat data frames as matrix data
#regions - list of regions/neighborhoods where demands come from (1 ... 210)
#locations -list of stations (1 ... 44)
#demands - hours x regions = demands
#regions = Int[parse(Int,string(x)) for x in names(hourly_calls[:,6:ncol(hourly_calls)])]
locations = collect(1:size(coverage,2))
coverage = Matrix(coverage)
coverage = convert(Array{Bool, 2}, coverage[:, :])
adjacent = Matrix(adjacent_nbhd[!,2:end]) 
adjacent = adjacent[regions,regions] .> 0.5
demand = Matrix(hourly_calls[!,5:end-1]); #we were trimming first 6 collumns but it seems to me it only the first contain extraneous data

In [8]:
# We focus on emergency calls during the "peak period" (8AM - 8PM),
# with the emergency calls from the first 3 month as our training set,
# and the subsequent emergency calls from the remaining months as our test set
peak_period = (hourly_calls[!,:hour] .>= 8) .* (hourly_calls[!,:hour] .<= 20)
indices = 1:DataFrames.nrow(hourly_calls);
train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
test_filter  = .~train_filter;
train_indices = indices[train_filter]
test_indices = indices[test_filter];

#will change the 30 later
p = DeploymentProblem(30, length(locations), length(regions), demand, train_indices,
                      test_indices, coverage[regions,:], Array{Bool,2}(adjacent));

In [9]:
amb_deployment = Dict{Symbol, Dict{Int, Vector{Int}}}()  
(deployment_model, name) = (next_dp -> StochasticDeployment(next_dp, nperiods=500), :Stochastic)
println("$name: ")
amb_deployment[name] = Dict{Int, Vector{Int}}()
namb = 39
println("$namb ")
p.nambulances = namb
println("time for model generation - stochastic $namb ambulances")

Stochastic: 
39 
time for model generation - stochastic 39 ambulances


In [25]:
using DelimitedFiles
for i in collect(35:45)
    namb = i
    println("$namb ")
    p.nambulances = namb
    println("time for model generation - stochastic $namb ambulances")
    @time model = deployment_model(p)
    set_optimizer(model.m, Gurobi.Optimizer)
    println("time for model solution - stochastic $namb ambulances")
    @time AmbulanceDeployment.optimize!(model, p)
    filename = "stoch_" * string(i) * ".csv"
    print(filename)
    writedlm(filename,  deployment(model), ',')
end

35 
time for model generation - stochastic 35 ambulances
Academic license - for non-commercial use only - expires 2022-07-26
 20.106328 seconds (56.54 M allocations: 

┌ Warning: Passing optimizer attributes as keyword arguments to
│ Gurobi.Optimizer is deprecated. Use
│     MOI.set(model, MOI.RawParameter("key"), value)
│ or
│     JuMP.set_optimizer_attribute(model, "key", value)
│ instead.
└ @ Gurobi C:\Users\Owner\.julia\packages\Gurobi\BAtIN\src\MOI_wrapper\MOI_wrapper.jl:273


4.489 GiB, 49.24% gc time)
Academic license - for non-commercial use only - expires 2022-07-26
time for model solution - stochastic 35 ambulances
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 126754 rows, 4702544 columns and 5902358 nonzeros
Model fingerprint: 0xbe5c9938
Variable types: 0 continuous, 4702544 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-06, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective 6644.0000000
Presolve removed 110348 rows and 4663502 columns
Presolve time: 3.89s
Presolved: 16406 rows, 39042 columns, 87909 nonzeros
Found heuristic solution: objective 2275.0043690
Variable types: 0 continuous, 39042 integer (22516 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   11485    4.3007027e+01   1.008500e+

┌ Warning: Passing optimizer attributes as keyword arguments to
│ Gurobi.Optimizer is deprecated. Use
│     MOI.set(model, MOI.RawParameter("key"), value)
│ or
│     JuMP.set_optimizer_attribute(model, "key", value)
│ instead.
└ @ Gurobi C:\Users\Owner\.julia\packages\Gurobi\BAtIN\src\MOI_wrapper\MOI_wrapper.jl:273


56.47% gc time)
Academic license - for non-commercial use only - expires 2022-07-26
time for model solution - stochastic 36 ambulances
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 126754 rows, 4702544 columns and 5902358 nonzeros
Model fingerprint: 0x6916660f
Variable types: 0 continuous, 4702544 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-06, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective 6644.0000000
Presolve removed 110348 rows and 4663502 columns
Presolve time: 3.81s
Presolved: 16406 rows, 39042 columns, 87909 nonzeros
Found heuristic solution: objective 2275.0043690
Variable types: 0 continuous, 39042 integer (22516 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   12694    2.9007108e+01   1.724000e+03   0.0000

...

 19.268971 seconds (56.54 M allocations: 4.489 GiB, 52.50% gc time)
Academic license - for non-commercial use only - expires 2022-07-26
time for model solution - stochastic 43 ambulances
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 126754 rows, 4702544 columns and 5902358 nonzeros
Model fingerprint: 0xae871fd7
Variable types: 0 continuous, 4702544 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-06, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective 6644.0000000
Presolve removed 110348 rows and 4663502 columns
Presolve time: 3.81s
Presolved: 16406 rows, 39042 columns, 87909 nonzeros
Found heuristic solution: objective 2275.0043690
Variable types: 0 continuous, 39042 integer (22516 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Ti

┌ Warning: Passing optimizer attributes as keyword arguments to
│ Gurobi.Optimizer is deprecated. Use
│     MOI.set(model, MOI.RawParameter("key"), value)
│ or
│     JuMP.set_optimizer_attribute(model, "key", value)
│ instead.
└ @ Gurobi C:\Users\Owner\.julia\packages\Gurobi\BAtIN\src\MOI_wrapper\MOI_wrapper.jl:273


 24.944374 seconds (56.54 M allocations: 4.489 GiB, 63.71% gc time)
Academic license - for non-commercial use only - expires 2022-07-26
time for model solution - stochastic 44 ambulances
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 126754 rows, 4702544 columns and 5902358 nonzeros
Model fingerprint: 0x164c6bd5
Variable types: 0 continuous, 4702544 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-06, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective 6644.0000000
Presolve removed 110348 rows and 4663502 columns
Presolve time: 3.75s
Presolved: 16406 rows, 39042 columns, 87909 nonzeros
Found heuristic solution: objective 2275.0043690
Variable types: 0 continuous, 39042 integer (22516 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Ti

┌ Warning: Passing optimizer attributes as keyword arguments to
│ Gurobi.Optimizer is deprecated. Use
│     MOI.set(model, MOI.RawParameter("key"), value)
│ or
│     JuMP.set_optimizer_attribute(model, "key", value)
│ instead.
└ @ Gurobi C:\Users\Owner\.julia\packages\Gurobi\BAtIN\src\MOI_wrapper\MOI_wrapper.jl:273


 19.009901 seconds (56.54 M allocations: 4.488 GiB, 52.58% gc time)
Academic license - for non-commercial use only - expires 2022-07-26
time for model solution - stochastic 45 ambulances
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 126754 rows, 4702544 columns and 5902358 nonzeros
Model fingerprint: 0x5efafc50
Variable types: 0 continuous, 4702544 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-06, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 6644.0000000
Presolve removed 110348 rows and 4663502 columns
Presolve time: 3.79s
Presolved: 16406 rows, 39042 columns, 87909 nonzeros
Found heuristic solution: objective 2275.0043690
Variable types: 0 continuous, 39042 integer (22516 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Ti

In [12]:
# old_x = [1, 1, 1, 2, 0, 0, 1, 2, 2, 0, 2, 0, 0, 1, 2, 2, 1, 2, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 2, 1]
# slack = [round(Int,x) for x in JuMP.value.(model2.slack)];
# x = [round(Int,x) for x in JuMP.value.(model2.x)];
# print(sum(x))
# print("\n")
# print(sum(slack))

In [4]:
# @time model2 = StochasticDeployment_rem(p, extra_amb = 1)
# set_optimizer(model2.m, Gurobi.Optimizer)
# println("time for model solution - stochastic $namb ambulances")
# @time AmbulanceDeployment.optimize!(model2, p)

In [21]:
#extra ambulance experiment
using DelimitedFiles
for i in collect(1:5)
    @time model2 = StochasticDeployment_hyp(p, extra_amb = i)
    set_optimizer(model2.m, Gurobi.Optimizer)
    println("time for model solution - stochastic $namb ambulances")
    @time AmbulanceDeployment.optimize!(model2, p)
    slack = [round(Int,x) for x in JuMP.value.(model2.slack)];
    x = [round(Int,x) for x in JuMP.value.(model2.x)];
    print(sum(x))
    print("\n")
    print(sum(slack))
    filename = "stoch_extra_amb_" * string(i) * ".csv"
    writedlm(filename,  x, ',')
end 

Academic license - for non-commercial use only - expires 2022-07-26


┌ Warning: Passing optimizer attributes as keyword arguments to
│ Gurobi.Optimizer is deprecated. Use
│     MOI.set(model, MOI.RawParameter("key"), value)
│ or
│     JuMP.set_optimizer_attribute(model, "key", value)
│ instead.
└ @ Gurobi C:\Users\Owner\.julia\packages\Gurobi\BAtIN\src\MOI_wrapper\MOI_wrapper.jl:273


 20.965177 seconds (56.54 M allocations: 4.489 GiB, 58.40% gc time)
Academic license - for non-commercial use only - expires 2022-07-26
time for model solution - stochastic 40 ambulances
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 126799 rows, 4702588 columns and 5902490 nonzeros
Model fingerprint: 0x5053c232
Variable types: 0 continuous, 4702588 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-06, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective 6644.0000000
Presolve removed 118839 rows and 4684822 columns (presolve time = 5s) ...
Presolve removed 118905 rows and 4684895 columns
Presolve time: 8.13s
Presolved: 7894 rows, 17693 columns, 40105 nonzeros
Found heuristic solution: objective 1437.0052070
Variable types: 0 continuous, 17693 integer (10607 binary)

Root simp

┌ Warning: Passing optimizer attributes as keyword arguments to
│ Gurobi.Optimizer is deprecated. Use
│     MOI.set(model, MOI.RawParameter("key"), value)
│ or
│     JuMP.set_optimizer_attribute(model, "key", value)
│ instead.
└ @ Gurobi C:\Users\Owner\.julia\packages\Gurobi\BAtIN\src\MOI_wrapper\MOI_wrapper.jl:273


 18.785399 seconds (56.54 M allocations: 4.489 GiB, 52.94% gc time)
Academic license - for non-commercial use only - expires 2022-07-26
time for model solution - stochastic 40 ambulances
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 126799 rows, 4702588 columns and 5902490 nonzeros
Model fingerprint: 0x44b3ce0d
Variable types: 0 continuous, 4702588 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-06, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective 6644.0000000
Presolve removed 118903 rows and 4684893 columns
Presolve time: 4.83s
Presolved: 7896 rows, 17695 columns, 40109 nonzeros
Found heuristic solution: objective 1436.0052080
Variable types: 0 continuous, 17695 integer (7188 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time

...

 21.149090 seconds (56.54 M allocations: 4.365 GiB, 54.65% gc time)
Academic license - for non-commercial use only - expires 2022-07-26
time for model solution - stochastic 40 ambulances
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 126799 rows, 4702588 columns and 5902490 nonzeros
Model fingerprint: 0xc53b9420
Variable types: 0 continuous, 4702588 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-06, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective 6644.0000000
Presolve removed 118902 rows and 4684892 columns
Presolve time: 4.31s
Presolved: 7897 rows, 17696 columns, 40111 nonzeros
Found heuristic solution: objective 1437.0052070
Variable types: 0 continuous, 17696 integer (7188 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time

┌ Warning: Passing optimizer attributes as keyword arguments to
│ Gurobi.Optimizer is deprecated. Use
│     MOI.set(model, MOI.RawParameter("key"), value)
│ or
│     JuMP.set_optimizer_attribute(model, "key", value)
│ instead.
└ @ Gurobi C:\Users\Owner\.julia\packages\Gurobi\BAtIN\src\MOI_wrapper\MOI_wrapper.jl:273


 19.408817 seconds (56.54 M allocations: 4.365 GiB, 54.44% gc time)
Academic license - for non-commercial use only - expires 2022-07-26
time for model solution - stochastic 40 ambulances
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 126799 rows, 4702588 columns and 5902490 nonzeros
Model fingerprint: 0xe620fe95
Variable types: 0 continuous, 4702588 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-06, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective 6644.0000000
Presolve removed 118902 rows and 4684892 columns
Presolve time: 3.89s
Presolved: 7897 rows, 17696 columns, 40111 nonzeros
Found heuristic solution: objective 1435.0052090
Variable types: 0 continuous, 17696 integer (7188 binary)

Root relaxation: objective 4.006640e+00, 5135 iterations, 0.03 seconds

    Nodes   

┌ Warning: Passing optimizer attributes as keyword arguments to
│ Gurobi.Optimizer is deprecated. Use
│     MOI.set(model, MOI.RawParameter("key"), value)
│ or
│     JuMP.set_optimizer_attribute(model, "key", value)
│ instead.
└ @ Gurobi C:\Users\Owner\.julia\packages\Gurobi\BAtIN\src\MOI_wrapper\MOI_wrapper.jl:273


 19.365617 seconds (56.54 M allocations: 4.366 GiB, 54.82% gc time)
Academic license - for non-commercial use only - expires 2022-07-26
time for model solution - stochastic 40 ambulances
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 126799 rows, 4702588 columns and 5902490 nonzeros
Model fingerprint: 0x58bbd43b
Variable types: 0 continuous, 4702588 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-06, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 6644.0000000
Presolve removed 118902 rows and 4684892 columns
Presolve time: 3.96s
Presolved: 7897 rows, 17696 columns, 40111 nonzeros
Found heuristic solution: objective 1435.0052090
Variable types: 0 continuous, 17696 integer (7188 binary)

Root relaxation: objective 3.006641e+00, 5041 iterations, 0.06 seconds

    Nodes   

In [12]:
if(false):
    @time model = deployment_model(p)
    set_optimizer(model.m, Gurobi.Optimizer)
    println("time for model solution - stochastic $namb ambulances")
    @time AmbulanceDeployment.optimize!(model, p)
if(true):
    @time model2 = StochasticDeployment_hyp(p, extra_amb = 1)
    set_optimizer(model2.m, Gurobi.Optimizer)
    println("time for model solution - stochastic $namb ambulances")
    @time AmbulanceDeployment.optimize!(model2, p)

LoadError: [91msyntax: line break in ":" expression[39m

In [22]:
#save julia variable as .csv
deployment(model)
#or
x_save = Array{Int64}(undef, 44)
count = 1
for x in JuMP.value.(model.x)
    x_save[count] = convert(Int,x)
    count = count + 1
end

using DelimitedFiles
writedlm("stoch_x.csv",  x_save, ',')
x_save

44-element Array{Int64,1}:
 1
 2
 1
 1
 1
 0
 1
 2
 2
 0
 1
 0
 0
 ⋮
 1
 2
 0
 0
 0
 0
 1
 0
 1
 0
 1
 1

#### Save the optimal routing. (Not stationing)

In [12]:
#save julia variable as .csv
I = size(model.y)[1]
J = size(model.y)[2]
K = size(model.y)[3]
y_save = Array{Int64}(undef, I,J,K)
for i in 1:I
    for j in 1:J
        for k in 1:K
            y_save[i,j,k] = convert(Int,JuMP.value.(model.y[i,j,k]))
        end
    end
end



In [13]:
using DelimitedFiles
writedlm( "stoch_y.csv",  y_save, ',')

In [14]:
print(sum(JuMP.value.(model.y[:,:,100])))

6.0

In [15]:
d = convert(DataFrame,y_save[:,:,1])

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
sum(y_save[:,:,2])


31

In [17]:
open("stoch_y.json","w") do f
    write(f, json_string)
end

LoadError: [91mUndefVarError: json_string not defined[39m

In [18]:
Pkg.add("NPZ")

  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`


In [19]:
using NPZ
npzwrite("stoch_y.npz",y_save)